<a href="https://colab.research.google.com/github/ssktotoro/neuro/blob/tutorial_branch/Neuro%20UNet%20Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuro UNet/ MeshnetTutorial

Authors: [Kevin Wang] (), [Alex Fedorov] (), [Sergey Kolesnikov](https://github.com/Scitator)

[![Catalyst logo](https://raw.githubusercontent.com/catalyst-team/catalyst-pics/master/pics/catalyst_logo.png)](https://github.com/catalyst-team/catalyst)

### Colab setup

First of all, do not forget to change the runtime type to GPU. <br/>
To do so click `Runtime` -> `Change runtime type` -> Select `\"Python 3\"` and `\"GPU\"` -> click `Save`. <br/>
After that you can click `Runtime` -> `Run all` and watch the tutorial.

## Requirements

Download and install the latest versions of catalyst and other libraries required for this tutorial.

In [3]:
%%bash 
git clone https://github.com/ssktotoro/neuro.git -b tutorial_branch
git pull
pip install -r neuro/requirements/requirements.txt


  Created wheel for python-multipart: filename=python_multipart-0.0.5-cp37-none-any.whl size=31671 sha256=9140de57ea6f4b9178b56ed40581a2fbdb68d8c10f8a24c4a040ebe7419adf5e
  Stored in directory: /root/.cache/pip/wheels/f0/e6/66/14a866a3cbd6a0cabfbef91f7edf40aa03595ef6c88d6d1be4
Successfully built python-multipart
  Found existing installation: idna 2.10
    Uninstalling idna-2.10:
      Successfully uninstalled idna-2.10
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


Cloning into 'neuro'...
fatal: not a git repository (or any of the parent directories): .git
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [4]:
from typing import Callable, List, Tuple
import numpy as np
import os
import torch
import catalyst
from catalyst import utils

print(f"torch: {torch.__version__}, catalyst: {catalyst.__version__}")

os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # "" - CPU, "0" - 1 GPU, "0,1" - MultiGPU

SEED = 42
utils.set_global_seed(SEED)
utils.prepare_cudnn(deterministic=True)

torch: 1.8.0+cu101, catalyst: 20.10.1


# Dataset

We'll be using the Mindboggle 101 dataset for a multiclass 3d segmentation task.
The dataset can be downloaded off osf with the following command from osfclient after you register with osf.

`osf -p 9ahyp clone .`

Otherwise you can download it using a Catalyst utility `download-gdrive` which downloads a version from the Catalyst Google Drive

`usage: download-gdrive {FILE_ID} {FILENAME}`

In [5]:
cd neuro

/content/neuro


In [6]:
%%bash
mkdir Mindboggle_data 
mkdir -p data/Mindboggle_101/
osf -p 9ahyp clone Mindboggle_data/
cp -r Mindboggle_data/osfstorage/Mindboggle101_volumes/ data/Mindboggle_101/
find data/Mindboggle_101 -name '*.tar.gz'| xargs -i tar zxvf {} -C data/Mindboggle_101
find data/Mindboggle_101 -name '*.tar.gz'| xargs -i rm {}

NKI-TRT-20_volumes/
NKI-TRT-20_volumes/NKI-TRT-20-7/
NKI-TRT-20_volumes/NKI-TRT-20-9/
NKI-TRT-20_volumes/NKI-TRT-20-8/
NKI-TRT-20_volumes/NKI-TRT-20-6/
NKI-TRT-20_volumes/NKI-TRT-20-1/
NKI-TRT-20_volumes/NKI-TRT-20-20/
NKI-TRT-20_volumes/NKI-TRT-20-18/
NKI-TRT-20_volumes/NKI-TRT-20-11/
NKI-TRT-20_volumes/NKI-TRT-20-16/
NKI-TRT-20_volumes/NKI-TRT-20-17/
NKI-TRT-20_volumes/NKI-TRT-20-10/
NKI-TRT-20_volumes/NKI-TRT-20-19/
NKI-TRT-20_volumes/NKI-TRT-20-4/
NKI-TRT-20_volumes/NKI-TRT-20-3/
NKI-TRT-20_volumes/NKI-TRT-20-2/
NKI-TRT-20_volumes/NKI-TRT-20-5/
NKI-TRT-20_volumes/NKI-TRT-20-15/
NKI-TRT-20_volumes/NKI-TRT-20-12/
NKI-TRT-20_volumes/NKI-TRT-20-13/
NKI-TRT-20_volumes/NKI-TRT-20-14/
NKI-TRT-20_volumes/NKI-TRT-20-14/labels.DKT31.manual.nii.gz
NKI-TRT-20_volumes/NKI-TRT-20-14/t1weighted_brain.MNI152.nii.gz
NKI-TRT-20_volumes/NKI-TRT-20-14/t1weighted.MNI152.nii.gz
NKI-TRT-20_volumes/NKI-TRT-20-14/labels.DKT31.manual+aseg.nii.gz
NKI-TRT-20_volumes/NKI-TRT-20-14/labels.DKT31.manual+aseg.MNI1

0files [00:00, ?files/s]
100%|██████████| 3.22M/3.22M [00:00<00:00, 184Mbytes/s]
1files [00:03,  3.65s/files]
100%|██████████| 3.66k/3.66k [00:00<00:00, 27.0Mbytes/s]
2files [00:04,  2.83s/files]
  0%|          | 0.00/843M [00:00<?, ?bytes/s]
  1%|          | 4.33M/843M [00:00<00:19, 43.1Mbytes/s]
  1%|          | 8.40M/843M [00:00<00:29, 28.8Mbytes/s]
  2%|▏         | 16.8M/843M [00:00<00:29, 28.2Mbytes/s]
  2%|▏         | 21.0M/843M [00:00<00:26, 31.3Mbytes/s]
  3%|▎         | 25.2M/843M [00:00<00:24, 33.8Mbytes/s]
  4%|▍         | 33.6M/843M [00:01<00:21, 38.5Mbytes/s]
  6%|▌         | 48.0M/843M [00:01<00:16, 49.3Mbytes/s]
  7%|▋         | 55.0M/843M [00:01<00:21, 37.3Mbytes/s]
  7%|▋         | 60.6M/843M [00:01<00:19, 39.4Mbytes/s]
  8%|▊         | 65.8M/843M [00:01<00:21, 35.3Mbytes/s]
  8%|▊         | 70.4M/843M [00:02<00:37, 20.7Mbytes/s]
  9%|▉         | 73.9M/843M [00:02<00:36, 21.0Mbytes/s]
  9%|▉         | 77.2M/843M [00:02<00:32, 23.5Mbytes/s]
 10%|▉         | 80.3M/843M [

Run the prepare data script that limits the labels to the DKT human labels (60 labels).

`usage: python ../neuro/scripts/prepare_data.py ../data/Mindboggle_101 {N_labels)`

In [7]:
%%bash 

python neuro/scripts/prepare_data.py data/Mindboggle_101/ 60

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


Import Catalyst and Torch utils for training

In [8]:
import torch
import collections

from catalyst.contrib.utils.pandas import read_csv_data
from torch.utils.data import RandomSampler, SequentialSampler
from torch.utils.data import DataLoader
from torchvision import transforms
from catalyst.data import Augmentor, ReaderCompose
from torch.optim.lr_scheduler import CosineAnnealingLR
from catalyst.dl import SupervisedRunner
from catalyst.callbacks.logging import TensorboardLogger
from catalyst.callbacks import SchedulerCallback, CheckpointCallback
from torchvision.transforms import ToTensor
from torch.nn import functional as F

In [9]:
from torch.nn import functional as F

### **Create the relevant Dataloaders for the specified train, validation, and inference BrainDatasets.**

BrainDatasets comprise of T1 scans + the prepared limited labels.

Training/ Validation batches: Randomly Sampled NxNxN Subvolumes from a Normal Distribution across the Volume Space with their corresponding labels.

Inference batches: Non-overlapping NxNxN Subvolumes across the existing volume space with their corresponding labels

More detail can be found in brain_dataset.py and generator_coords.py  

In [38]:
cd training/

/content/neuro/training


In [28]:
from brain_dataset import BrainDataset
from reader import NiftiReader_Image, NiftiReader_Mask
from callbacks import CustomDiceCallback
from model import UNet, MeshNet
from custom_metrics import custom_dice_metric

In [29]:
open_fn = ReaderCompose(                                                                                                                                                                            
    readers=[                                                                                                                                                                                       
        NiftiReader_Image(input_key="images", output_key="images"),                                                                                                                                 
        NiftiReader_Mask(input_key="nii_labels", output_key="targets"),
    ]
)

In [30]:

def get_loaders(
    random_state: int,
    volume_shape: List[int],
    subvolume_shape: List[int],
    in_csv_train: str = None,                                                                                                                                                                           
    in_csv_valid: str = None,                                                                                                                                                                           
    in_csv_infer: str = None,
    batch_size: int = 16,
    num_workers: int = 10,
) -> dict:

    df, df_train, df_valid, df_infer = read_csv_data(                                                                                                                                                   
    in_csv_train=in_csv_train,                                                                                                                                                                      
    in_csv_valid=in_csv_valid,                                                                                                                                                                      
    in_csv_infer=in_csv_infer,                                                                                                                                                                      
    ) 

    datasets = {}

    train_dataset = BrainDataset(shared_dict={},                                                                                                                                                             
                    list_data=df_train, list_shape=volume_shape, list_sub_shape=subvolume_shape,                                                                                                              
                    open_fn=open_fn,                                                                                                         
                    mode='train', input_key="images",                                                                                                                                     
                    output_key="targets")
    valid_dataset = BrainDataset(shared_dict={},                                                                                                                                                             
                    list_data=df_valid, list_shape=volume_shape, list_sub_shape=subvolume_shape,                                                                                                              
                    open_fn=open_fn,                                                                                                         
                    mode='validation', input_key="images",                                                                                                                                     
                    output_key="targets")
    test_dataset = BrainDataset(shared_dict={},                                                                                                                                                             
                    list_data=df_infer, list_shape=volume_shape, list_sub_shape=subvolume_shape,                                                                                                              
                    open_fn=open_fn,                                                                                                         
                    mode='infer', input_key="images",                                                                                                                                     
                    output_key="targets")

    train_random_sampler = RandomSampler(data_source=train_dataset,                                                                                                                                   
                                          replacement=True,
                                          num_samples=len(train_dataset) * 16)

    valid_random_sampler = RandomSampler(data_source=valid_dataset,  
                                          replacement=True,
                                          num_samples=len(valid_dataset)*16)
    
    infer_random_sampler = SequentialSampler(data_source=test_dataset)

    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, sampler=train_random_sampler, 
                              num_workers=2, pin_memory=True)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, sampler=valid_random_sampler, 
                              num_workers=2, pin_memory=True,drop_last=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=1, sampler=infer_random_sampler, 
                              num_workers=2, pin_memory=True,drop_last=True)
    train_loaders = collections.OrderedDict()
    infer_loaders = collections.OrderedDict()
    train_loaders["train"] = train_loader
    train_loaders["valid"] = valid_loader
    infer_loaders['infer'] = test_loader

    return train_loaders, infer_loaders

In [39]:
train_loaders, infer_loaders = get_loaders(0, [256, 256, 256], [38, 38, 38], 
                      "../data/dataset_train.csv", "../data/dataset_valid.csv", "../data/dataset_infer.csv", )

# Model Training

We'll train the model 1 epoch for demonstration although typically we train for 30 epochs.

* Scheduler: Adam with a cosine annealing schedule starting at a learning rate of .01 
* Batch Metric: DICE
* Loss: CrossEntropyLoss
* Logger: Tensorboard
* CheckpointerCallback

For training and validation we sample the volume with subvolumes specified in our Dataset 

In [40]:
cd ..

/content/neuro


In [16]:
class CustomRunner(catalyst.dl.Runner):

    def predict_batch(self, batch):
        # model inference step
        return self.model(batch['images'].to(self.device)), batch['coords']

    def _handle_batch(self, batch):
        # model train/valid step
        x, y = batch['images'], batch['targets']
        y_hat = self.model(x)

        loss = F.cross_entropy(y_hat, y)
        self.batch_metrics.update({"loss": loss, "dice": custom_dice_metric(y_hat.float(), y, num_classes=60, activation='Softmax')})

        if self.is_train_loader:
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

In [17]:
n_classes = 60
torch.backends.cudnn.deterministic = False
meshnet = MeshNet(n_channels=1, n_classes=n_classes)

logdir = "logs/meshnet"

optimizer = torch.optim.Adam(meshnet.parameters(), lr=0.01, weight_decay=0.0001)
scheduler = CosineAnnealingLR(optimizer, T_max=30)

runner = CustomRunner()
runner.train(model=meshnet, optimizer=optimizer, loaders=train_loaders, num_epochs=1, scheduler=scheduler, 
             callbacks=[TensorboardLogger(), CheckpointCallback()], logdir=logdir, verbose=True)

1/1 * Epoch (train):   0% 0/80 [00:00<?, ?it/s]

/content/neuro/training/custom_metrics.py:16: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



1/1 * Epoch (train): 100% 80/80 [09:50<00:00,  7.38s/it, dice=0.992, loss=0.009]
1/1 * Epoch (valid): 100% 20/20 [02:21<00:00,  7.06s/it, dice=0.978, loss=0.024]
[2021-03-27 03:44:22,893] 
1/1 * Epoch 1 (train): dice=0.7337 | loss=0.7947
1/1 * Epoch 1 (valid): dice=0.9808 | loss=0.0206
Top best models:
logs/meshnet/checkpoints/train.1.pth	0.0206


In [47]:
def voxel_majority_predict_from_subvolumes(loader, volume_shape, n_classes):
  segmentations = {}
  for subject in range(loader.dataset.subjects):
    segmentations[subject] = torch.zeros(tuple(np.insert(volume_shape, 0, n_classes)), dtype=torch.uint8)

  for inference in runner.predict_loader(loader=loader):
    subj_id = loader.dataset.subjects // len(loader.dataset.coords)
    coords = inference[1]
    _, predicted = torch.max(log_softmax(inference[0].cpu(), dim=1), 1)
    for j in range(predicted.shape[0]):
      c_j = coords[j][0]
      for c in range(n_classes):
        segmentations[subj_id][c, c_j[0, 0]:c_j[0, 1], 
                              c_j[1, 0]:c_j[1, 1], 
                              c_j[2, 0]:c_j[2, 1]] += (predicted[j] == c)

  for i in segmentations.keys():
    segmentations[i] = torch.max(segmentations[i], 0)[1]
  return segmentations

In [26]:
from torch.nn.functional import log_softmax

In [60]:

segmentations = {}
for subject in range(infer_loader.dataset.subjects):
  segmentations[subject] = torch.zeros(tuple(np.insert([256, 256, 256], 0, 60)), dtype=torch.uint8)

for inference in runner.predict_loader(loader=infer_loader):
  subj_id = infer_loader.dataset.subjects // len(infer_loader.dataset.coords)
  coords = inference[1]
  print(coords)
  _, predicted = torch.max(log_softmax(inference[0].cpu(), dim=1), 1)
  for j in range(predicted.shape[0]):
    c_j = coords[j][0]
    for c in range(n_classes):
      segmentations[subj_id][c, c_j[0, 0]:c_j[0, 1], 
                            c_j[1, 0]:c_j[1, 1], 
                            c_j[2, 0]:c_j[2, 1]] += (predicted[j] == c)

for i in segmentations.keys():
  segmentations[i] = torch.max(segmentations[i], 0)[1]


tensor([[[[14, 52],
          [14, 52],
          [14, 52]]]])
tensor([[[[14, 52],
          [14, 52],
          [52, 90]]]])
tensor([[[[ 14,  52],
          [ 14,  52],
          [ 90, 128]]]])
tensor([[[[ 14,  52],
          [ 14,  52],
          [128, 166]]]])
tensor([[[[ 14,  52],
          [ 14,  52],
          [166, 204]]]])
tensor([[[[ 14,  52],
          [ 14,  52],
          [204, 242]]]])
tensor([[[[14, 52],
          [52, 90],
          [14, 52]]]])
tensor([[[[14, 52],
          [52, 90],
          [52, 90]]]])
tensor([[[[ 14,  52],
          [ 52,  90],
          [ 90, 128]]]])
tensor([[[[ 14,  52],
          [ 52,  90],
          [128, 166]]]])
tensor([[[[ 14,  52],
          [ 52,  90],
          [166, 204]]]])
tensor([[[[ 14,  52],
          [ 52,  90],
          [204, 242]]]])
tensor([[[[ 14,  52],
          [ 90, 128],
          [ 14,  52]]]])
tensor([[[[ 14,  52],
          [ 90, 128],
          [ 52,  90]]]])
tensor([[[[ 14,  52],
          [ 90, 128],
          [ 90

In [59]:
next(iter(infer_loader))

{'coords': tensor([[[[14, 52],
           [14, 52],
           [14, 52]]]]),
 'images': tensor([[[[[0.0017, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0017, 0.0017, 0.0017,  ..., 0.0017, 0.0000, 0.0000],
            ...,
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0017],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0017, 0.0017],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
           [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0017, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            ...,
            [0.0000, 0.0000, 0.0000,  ..., 0.0017, 0.0017, 0.0000],
            [0.0000, 0.0017, 0.0000,  ..., 0.0017, 0.0000, 0.0017],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
           [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000

In [48]:
n_classes = 60
infer_loader = infer_loaders['infer']

segmentations = voxel_majority_predict_from_subvolumes(infer_loader, [256, 256, 256], n_classes)

In [55]:
segmentations[0].max()

tensor(0)